Reading xml files

In [5]:
import xml.etree.ElementTree as ET
import os
import pandas as pd

#  file path is to a folder nemde in the users $HOME folder
#  keeping data in here allows portability to any UNIX machine \
#   - including AWS Linux instances
path = '{}/nem-data/nemde/_20090701/NEMPriceSetter_2009070100100.xml'.format(os.environ['HOME'])
#xml are hierachical data represented as tree
tree = ET.parse(path)
#root is highest hierachy
root = tree.getroot()
#here, command iterates over all children nodes of root
data = [c.attrib for c in root]

print(type(data))

df = pd.DataFrame(data)
df.query('Market == "Energy" and RegionID == "SA1"')

<class 'list'>


,PeriodID,RegionID,Market,Price,Unit,DispatchedMarket,BandNo,Increase,RRNBandPrice,BandCost
36,2009-07-01T04:05:00+10:00,SA1,Energy,1.35918,YWPS2,ENOF,2,1.32679,4.23000,5.61232
37,2009-07-01T04:05:00+10:00,SA1,Energy,1.35918,ER01,ENOF,2,-0.08861,12.00000,-1.06333
38,2009-07-01T04:05:00+10:00,SA1,Energy,1.35918,"ENOF,YWPS2,2,YWPS3,2",tbslack1,NaN,185.75049,0.00000,0.00019
39,2009-07-01T04:05:00+10:00,SA1,Energy,1.35918,ER04,ENOF,2,-0.08861,12.00000,-1.06333
40,2009-07-01T04:05:00+10:00,SA1,Energy,1.35918,ER03,ENOF,2,-0.08861,12.00000,-1.06333
41,2009-07-01T04:05:00+10:00,SA1,Energy,1.35918,ER02,ENOF,2,-0.08861,12.00000,-1.06333


Scraping NEM data.   First you get lists of URLs 
- this needs to be done multiple times - to go from years to months to .zip's

In [15]:
import requests
import bs4
import os

url = 'http://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2019'
#allows to send http/1.1 requests extremely easily
req = requests.get(url)

#status code 200 means 'OK' within 'Success'; analogous to '404 Not Found'
assert req.status_code == 200

#.encoding e.g. utf-8; BeautifulSoup to pull data from html, xml; extracts DOM
soup = bs4.BeautifulSoup(req.content, 'html.parser')

#  find all links in the page wihin 'a' class
links = soup.find_all('a', href=True)

#  create the url for each month
links = [os.path.join(url, link.get_text()) #INTELLIGENTLY joins path components
         for link in links]



#  remove the first as this the link to parent directory
#  show two
links[1:][:2]

['http://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2019/MMSDM_2019_01',
 'http://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2019/MMSDM_2019_01.zip']

After you have to the url, use `requests` to download the .zip & the unzip using Python s
- needs some documenting

In [20]:
#  note that

import logging
import sys
import zipfile

#extract_zip_from_url(url, os.environ['HOME']); .zip download-url, /home/bastian
def extract_zip_from_url(url, output_path):
    #take month-specific appendix of zip file
    month = url[-17:-4]
    #folder path (e.g. '/home/bastian/_201907010000')
    output_folder = os.path.join(output_path, month)
    #Recursive directory creation function. Makes all intermediate-level directories needed to contain the leaf directory.
    #Here: creates folder with name according to 'output folder'
    #If exist_ok is False (the default), an FileExistsError is raised if the target directory already exists.
    os.makedirs(output_folder, exist_ok=True)

    #block to download urls
    #info message
    logging.info('downloading {}'.format(url))
    #mms is path with mmsdm.zip appendix
    mms = os.path.join(output_folder, 'mmsdm.zip')
    #the following creates (zip) file with name of last segment of mms path
    with open(mms, 'wb') as f:
        #sends get request and returns status code
        response = requests.get(url, headers=header, stream=True)
        #The Content-Length header is a number denoting an the exact byte length of the HTTP body
        #DOM variable!?
        total_length = response.headers.get('content-length')

        if total_length is None: # no content length header
            #simply write encoding
            f.write(response.content)
        else:
            dl = 0
            total_length = int(total_length)
            #some sort of data chunks
            for data in response.iter_content(chunk_size=4096):
                dl += len(data)
                f.write(data)
                done = int(50 * dl / total_length)
                sys.stdout.write('\r {}% [{}{}]'.format(2*done, '+'*done, ' '*(50-done)))
                sys.stdout.flush()

    logging.info('extracting {}'.format(url))
    
    #unzipping of zip from previous block
    with zipfile.ZipFile(mms, 'r') as my_zipfile:
        my_zipfile.extractall(output_folder)

header = None
url = 'http://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2019/MMSDM_2019_07/MMSDM_Historical_Data_SQLLoader/DATA/PUBLIC_DVD_DISPATCH_UNIT_SCADA_201907010000.zip'
extract_zip_from_url(url, '{}/nem-data/xml_raw'.format(os.environ['HOME']))

 100% [++++++++++++++++++++++++++++++++++++++++++++++++++]

In [20]:
!ls ~/nem-data/raw-mms

MMSDM_2018_01 MMSDM_2018_03 MMSDM_2018_05 MMSDM_2018_07
MMSDM_2018_02 MMSDM_2018_04 MMSDM_2018_06 log.log


In [22]:
!ls

README.md notebooks reference


In [19]:
os.environ['HOME']

'/home/bastian'